

# Propaganda, start the `spark` session

> For SQL users, Spark SQL provides state-of-the-art SQL performance and maintains compatibility with Shark/Hive. In particular, like Shark, Spark SQL supports all existing Hive data formats, user-defined functions (UDF), and the Hive metastore.

> For Spark users, Spark SQL becomes the narrow-waist for manipulating (semi-) structured data as well as ingesting data from sources that provide schema, such as JSON, Parquet, Hive, or EDWs. It truly unifies SQL and sophisticated analysis, allowing users to mix and match SQL and more imperative programming APIs for advanced analytics.

> For open source hackers, Spark SQL proposes a novel, elegant way of building query planners. It is incredibly easy to add new optimizations under this framework.

> Internally, a structured query is a Catalyst tree of (logical and physical) relational operators and expressions.




In [ ]:
# import the usual suspects
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
from pathlib import Path
import sys
import timeit

%matplotlib inline
import seaborn as sns

sns.set_context("notebook", font_scale=1.2)

During the session, we will use classes and functions exported by `pyspark`


In [ ]:
# spark
from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession
from pyspark.sql import Window
from pyspark.sql.functions import col
import pyspark.sql.functions as fn
from pyspark.sql.catalog import Catalog
from pyspark.sql.types import StructType, StructField
from pyspark.sql.types import IntegerType, StringType

Start the `SparkSession`

In [ ]:
conf = SparkConf().setAppName("Spark SQL Illustrations")
sc = SparkContext(conf=conf)

spark = (SparkSession
    .builder
    .appName("Spark SQL")
    .getOrCreate()
)

US Baby Names 1880-2017
=======================


Description
: US baby names provided by the SSA. 

This dataset contains all names used
for at least 5 children of either sex during a year. 


The file is made of `1924665` lines and  4 columns.

```
|-- name: string (nullable = true)
    |-- n: integer (nullable = true)
    |-- sex: string (nullable = true)
    |-- year: integer (nullable = true)
```

Each row indicates for a given name, sex, and year the number of babies 
of the given sex who were given that name during the given year. Names 
with less than 5 occurrences during the year were note recorded. 

|    name|  n|sex|year|
|:--------|:---:|:---:|:----:|
|  Emilia|112|  F|1985|
|   Kelsi|112|  F|1985|
|  Margot|112|  F|1985|
|  Mariam|112|  F|1985|
|Scarlett|112|  F|1985|

First, we download the data if it's not there yet

In [ ]:
import requests, zipfile, io
from pathlib import Path

path = Path('babynames_short.csv')
if not path.exists():
    url = "https://stephanegaiffas.github.io/big_data_course/data/babynames_short.csv.zip"
    r = requests.get(url)
    z = zipfile.ZipFile(io.BytesIO(r.content))
    z.extractall(path='./')

Load `babynames` from a `csv` file

In [ ]:
df_sp = spark.read\
             .format('csv')\
             .option("header", "true")\
             .option("mode", "FAILFAST")\
             .option("inferSchema", "true")\
             .option("sep", ",")\
             .load("babynames_short.csv")

df_sp.printSchema()


Ensure that the dataframe has the following schema:

    root
        |-- name: string (nullable = true)
        |-- n: integer (nullable = true)
        |-- sex: string (nullable = true)
        |-- year: integer (nullable = true)




In [ ]:
df_sp = df_sp.withColumn("year", df_sp.year.cast("int"))
df_sp.printSchema()

SQL versus spark-Dataframe API
=================================

>  Dataset API vs SQL

> Spark SQL supports two "modes" to write structured queries: Dataset API and SQL. SQL Mode is used to express structured queries using SQL statements using SparkSession.sql operator, expr standard function and spark-sql command-line tool.

> Some structured queries can be expressed much easier using Dataset API, but there are some that are only possible in SQL. In other words, you may find mixing Dataset API and SQL modes challenging yet rewarding.

> What is important, and one of the reasons why Spark SQL has been so successful, is that there is no performance difference between the modes. Whatever mode you use to write your structured queries, they all end up as a tree of Catalyst relational data structures. And, yes, you could consider writing structured queries using Catalyst directly, but that could quickly become unwieldy for maintenance (i.e. finding Spark SQL developers who could be comfortable with it as well as being fairly low-level and therefore possibly too dependent on a specific Spark SQL version).

Warmup:  compute the 10 most popular names given to babies in year 2000.
======================================================================

## Using `spark.sql()`

In order to use mode `sql`, create a temporary view from the `DataFrame`.

1. What are temporary views made of?
1. Are there other kind of views in spark's world?

In [ ]:
# 1. It's a lazily evaluated "view". In SQL, a view is a virtual table based on the result-set of an SQL statement.
#    Temporary views in Spark SQL are session-scoped and will disappear if the session that creates it terminates.
#    It does not persist in memory unless you cache the dataset that underpins the view. 

# 2. The only other "type" of view in the scala world is the Global Temporary view, but it's the same as the former
 #   but "global" to the application and not only the session
Catalog(spark).listTables()
# Empty

In [ ]:
df = df_sp
df.createOrReplaceTempView("babynames")

Catalog(spark).listTables()


## A query is a plain SQL query embodied in a string.



In [ ]:
query = """
SELECT * FROM babynames
WHERE (year = {0} AND sex = {1})
ORDER BY n DESC
LIMIT 10
"""

spark.sql(query.format(2000, "'M'")).union(spark.sql(query.format(2000, "'F'"))).show()


> This phrasing is not consistent with the DRY principle. Fix this using formatted strings.

## Using the dataframe/dataset API

This can also be done using Spark SQL API.

### Pedestrian approach

1. First select `10` most popular names for girls in year `2000`, define `spark` dataframe
`top10_2000_f`.
1. Does the definition of `top10_2000_f` involve _transformations_, _actions_ or both?
1. What is the type of the result returned by `top10_2000_f.take(2)`? the type of elements of the result?



In [ ]:
top10_2000_f = df.where("sex = 'F'").where("year = 2000").orderBy(df.n.desc()).limit(10)
top10_2000_f.show()
type(top10_2000_f.take(2)) # It's a 'list'

In [ ]:
type(top10_2000_f.take(2)[0]) #pyspark.sql.types.Row


1. Do the same thing for boys.



In [ ]:
top10_2000_m = df.where("sex = 'M'").where("year = 2000").orderBy(df.n.desc()).limit(10)
top10_2000_m.show()


1. Compute the _union_ of the two spark dataframes. Store the result in
dataframe `top10_2000`



In [ ]:
top10_2000 = top10_2000_f.union(top10_2000_m)
top10_2000.show()


### Do it again, complying  with DRY principle



In [ ]:
temp_df = df.where("0 = 1")
genders = ["'M'", "'F'"]
for g in genders: 
    temp_df = temp_df.union(df.where("year = 2000").where("".join(["sex = ", g])).orderBy(df.n.desc()).limit(10))
temp_df.show()



Name portfolio through ages
===========================

1. Compute for each year and sex the number of distinct names given that year.



In [ ]:
nb_names_year_sex = df.orderBy(df.year.asc()).groupBy('year', 'sex').count()
window = Window.partitionBy('year')
count = fn.sum('count').over(window)
nb_names_year_sex = nb_names_year_sex.withColumn('total_count', count)
nb_names_year_sex.show()


1. Plot the evolution of the number of distinct names as a function of `year`.
Use some aesthetics to distinguish sexes.




In [ ]:
pd_df = nb_names_year_sex.toPandas()
pd_df
p = sns.lineplot(data=pd_df, x='year', y='count', hue='sex')
p.set_title("Number of distinct names with more than 5 occurrences")

plt.show(p)



Assessing popularity through time
=================================

1. For each year and sex, compute the total number of births
1. Plot the evolution of the sex ratio over time
1. For each year, sex, and name compute the percentage of newborns
given that name for that given year.


> Use `Window` functions.



In [ ]:
births = df.withColumn('births', fn.sum('n').over(Window.partitionBy('year', 'sex').orderBy('year'))) \
    .groupBy('year', 'sex', 'births').count() \
    .select('year', 'sex', 'births') \
    .withColumn('total', fn.sum('births').over(Window.partitionBy('year').orderBy('year')))
births.orderBy('year', 'sex').show(20)

In [ ]:
df_sex_ratio = births.groupBy("year") \
    .pivot("sex") \
    .sum("births") \
    .withColumn("sex_ratio", fn.round(col("M")/col("F"), 2)) \
    .orderBy("year").select("year", "M", "F", "sex_ratio")
df_sex_ratio.show(5)

q = sns.lineplot(data=df_sex_ratio.toPandas(), x='year', y='sex_ratio')
q.set_title("Evolution of the sex ratio over time")
plt.show(q)


1. Compute for each year, sex and name  the `row_number`, `rank`, and `dense_rank`
of the name within that year and sex category, when names are sorted by increasing popularity.



In [ ]:
window = Window.partitionBy('year', 'sex').orderBy(df.n.desc())
rank = fn.rank().over(window)
dense_rank = fn.dense_rank().over(window)
row_number = fn.row_number().over(window)

rank_df = df.withColumn('row_number', row_number) \
    .withColumn('rank', rank) \
    .withColumn('dense_rank', dense_rank)
rank_df.orderBy('year', rank_df.n.desc()).show()




Evolution of top popular names through the century
==================================================


1. For each sex, select the ten most popular names in year 2000, and plot the proportion
of newborns given that name over time. Take into account that some names might have
zero occurrence during certain years.



In [ ]:
popular_df = rank_df.where('year = 2000') \
    .where("dense_rank <= 10")
popular_df.show()
popular_over_time = popular_df.select('name', 'sex') \
    .join(df, ['name', 'sex'], "left") \
    .join(births.where('sex = "M"').select('year', 'total'), 'year', 'left')
popular_over_time = popular_over_time.withColumn('proportion', popular_over_time['n'] / popular_over_time['total'])
popular_over_time.show(20)


1. Use `explain()` to determine the joining strategy used by spark.



In [ ]:
popular_over_time.explain()

Plot  the popularity of each of the top ten achievers from year 2000 with respect to time
==================================================================================

In [ ]:
pd_pot = popular_over_time.toPandas()
sns.set(rc={'figure.figsize':(12,8)}, style='whitegrid')
q = sns.lineplot(data=pd_pot, x='year', y='proportion', hue='name', style='sex')
q.set_title("Popularity of each of the top ten names from year 2000 with respect to time ")
plt.show(q)



Plot the total popularity of the top ten achievers from year 2000 with respect to time
==================================================================================




In [ ]:
pot = popular_over_time.withColumn('total_proportion', fn.sum('proportion').over(Window.partitionBy('year')))
pot = pot.select('year', 'total_proportion')
sns.set(rc={'figure.figsize':(12,6)}, style='whitegrid')
q = sns.lineplot(data=pot.toPandas(), x='year', y='total_proportion')
q.set_title("Total popularity of the top ten names from year 2000 as a function of time")
plt.show(q)

 
Plot lorenz curves
=====================

Every year, the name counts define a discrete probability distribution.
This distribution, just as income or wealth distribution,
is (usually) far from being uniform. We want to assess how uneven it is.
We use the tools developed in econometrics.

Without loss of generality, that we handle a distribution over $1, \ldots, n$
where $n$ is the number of distinct names given during a year.
We assume that frequencies $p_1, p_2, \ldots, p_n$ are given in ascending order.

The Lorenz function maps $[0, 1] \to [0, 1]$.
$$L(x) = \sum_{i=1}^{\lfloor nx \rfloor} p_i$$.

1. Design a query that adds a column "lorenz" to the dataframe , and for each
row computes the value of the Lorenz function defined by `year`  and `sex`.




In [ ]:
lor = df.withColumn('births', fn.sum('n').over(Window.partitionBy('year', 'sex').orderBy('year'))) \
    .groupBy('year', 'sex', 'births').count() \
    .select('year', 'sex', 'births') \
    .join(df, ['year', 'sex'], "right") \
    .withColumn('annual_proportion', col('n')/col('births')) \
    .withColumn('lorenz', fn.sum('annual_proportion') \
                .over(Window.partitionBy('year', 'sex')\
                .orderBy('annual_proportion') \
                .rangeBetween(Window.unboundedPreceding, 0))) \
    .withColumn('row', fn.row_number().over(Window.partitionBy('year', 'sex')\
                .orderBy('annual_proportion'))) \
    .withColumn('normalized_row', col('row') / fn.max('row').over(Window.partitionBy('year', 'sex')))
lor.orderBy(lor.year.desc(), lor.sex.asc(), lor.lorenz.desc()).show()
lor = lor.drop('births', 'n', 'births')
lor.show()

In [ ]:
pd_lor = lor.where("year = 2017").toPandas()
sns.lineplot(data=pd_lor, x='normalized_row', y='lorenz', hue='sex')
q = sns.lineplot(x=[0,1], y=[0,1], color='green', dashes=True, label='x = y')
q.set_title("Normalized Lorenz function")
plt.show(q)



1. Design a function that takes as input a `year` and plots the Lorenz curve
for that year for both sexes.



In [ ]:
query = lambda y: df.where('year = {}'.format(y)) \
    .withColumn('births', fn.sum('n').over(Window.partitionBy('sex'))) \
    .groupBy('year', 'sex', 'births').count() \
    .select('year', 'sex', 'births') \
    .join(df, ['year', 'sex'], "left") \
    .withColumn('annual_proportion', col('n')/col('births')) \
    .withColumn('lorenz', fn.sum('annual_proportion') \
                .over(Window.partitionBy('sex')\
                .orderBy('annual_proportion') \
                .rangeBetween(Window.unboundedPreceding, 0))) \
    .withColumn('row', fn.row_number().over(Window.partitionBy('sex')\
                .orderBy('annual_proportion'))) \
    .withColumn('normalized_row', col('row') / fn.max('row').over(Window.partitionBy('sex')))

def lorenz_plot(year):
    lorenz_df = query(year)
    sns.lineplot(data=lorenz_df.toPandas(), x='normalized_row', y='lorenz', hue='sex')
    q = sns.lineplot(x=[0,1], y=[0,1], color='green', dashes=True, label='x = y')
    q.set_title("Lorenz function during the year "+str(year))
    plt.show(q)

In [ ]:
lorenz_plot(1890)


Gini index
==========

The [Gini index](https://en.wikipedia.org/wiki/Gini_coefficient) is twice the surface of the area comprised between curves $y=x$
and $y=L(x)$.

Choose a formula that allows you to compute it efficiently.

$$G={\frac {2\sum _{i=1}^{n}iy_{i}}{n\sum _{i=1}^{n}y_{i}}}-{\frac {n+1}{n}}.$$

1. Design a query that computes the Gini index of the `babynames` distribution
for every `year` and `sex`.

1. Plot Gini index over time




In [ ]:
iy_i =  col("row")*col("annual_proportion")
index_max = fn.max("row").over(Window.partitionBy("year", "sex"))
den = index_max*fn.sum("annual_proportion").over(Window.partitionBy("year", "sex"))
nb_births = fn.sum('n').over(Window.partitionBy('year', 'sex').orderBy('year'))

gini_df = df.withColumn("row", fn.row_number().over(Window.partitionBy("year", "sex").orderBy("n"))) \
    .withColumn('annual_proportion', col('n')/nb_births) \
    .withColumn("gini", (2*fn.sum(iy_i).over(Window.partitionBy("year", "sex"))/den ) - (index_max+1)/index_max)

gini_df.show()

In [ ]:
gini = gini_df.toPandas()
q = sns.lineplot(x="year", y="gini", data=gini, hue="sex")
q.set_title("Gini index over time")
plt.show(q)




Close the door, leave work area clean
=====================================


In [ ]:
spark.stop()